The Perceptron

In [52]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

iris = load_iris()

X = iris.data[:, (2, 3)] # petal length and petal width
y = (iris.target == 0).astype(np.int)

per_clf = Perceptron(random_state=42)
per_clf.fit(X, y)

X_pred = per_clf.predict([[2, 0.5]])
print(X_pred)

[1]


/home/dimension/.virtualenvs/data-science/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [53]:
import os
from six.moves import urllib
from scipy.io import loadmat

mnist_path = os.path.join("/home/dimension/", "scikit_learn_data/mldata", "mnist-original.mat")

if not os.path.isfile(mnist_path):
    # # download dataset from github.
    mnist_alternative_url = "https://github.com/amplab/datascience-sp14/raw/master/lab7/mldata/mnist-original.mat"
    response = urllib.request.urlopen(mnist_alternative_url)
    with open(mnist_path, "wb") as f:
        content = response.read()
        f.write(content)

mnist_raw = loadmat(mnist_path)
mnist = {
    "data": mnist_raw["data"].T,
    "target": mnist_raw["label"][0],
    "COL_NAMES": ["label", "data"],
    "DESCR": "mldata.org dataset: mnist-original",
}
print("Done!")

Done!


Training an MLP with TensorFlow's High-Level API

In [6]:
import os
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

X, y = mnist["data"], mnist["target"]

scaler = StandardScaler()
scaler.fit(X)
y = y.astype(np.int)

# MNIST dataset is actually already split into training(first 60,000 images) and test set (10,000 images)
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:] 

feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300, 100], n_classes=10, feature_columns=feature_cols)
dnn_clf.fit(X_train, y_train, batch_size=50, steps=4000)

y_pred = list(dnn_clf.predict(X_test))
accuracy_score(y_test, y_pred)


/home/dimension/.virtualenvs/data-science/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpow_e53kq', '_log_step_count_steps': 100, '_num_worker_replicas': 0, '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2422f85a90>, '_tf_random_seed': None, '_train_distribute': None, '_master': '', '_keep_checkpoint_max': 5, '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_evaluation_master': '', '_task_type': None, '_environment': 'local', '_num_ps_replicas': 0, '_save_checkpoints_secs': 600, '_task_id': 0, '_session_config': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpow_e53kq/model.ckpt.
INFO:tensorflow:step = 1, loss = 131.1536
INFO:tensorfl

0.914

Training a DNN using plain TensorFlow

Construction Phase

In [54]:
tf.reset_default_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

Creating Neural Network

In [55]:
def neuron_layer(input_X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(input_X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(input_X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z
        
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation = tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

Creating Neural Network using TensorFlow Library

In [56]:
with tf.name_scope("dnn_tf"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

Calculating Loss

In [57]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

Training Model

In [58]:
learning_rate = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

Execution Phase

In [91]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data")



Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [43]:
n_epochs = 40
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iterations in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
            
        acc_train = accuracy.eval(feed_dict={X:X_batch, y:y_batch})
        acc_test = accuracy.eval(feed_dict={X:mnist.test.images, y:mnist.test.labels})
        
        print("Epoch: ", epoch, " Train Accuracy: ", acc_train, " Test Accuracy: ", acc_test)
    
    save_path = saver.save(sess, "./my_model_final.ckpt")

Epoch:  0  Train Accuracy:  0.92  Test Accuracy:  0.9144
Epoch:  1  Train Accuracy:  0.96  Test Accuracy:  0.9329
Epoch:  2  Train Accuracy:  0.94  Test Accuracy:  0.9395
Epoch:  3  Train Accuracy:  0.9  Test Accuracy:  0.9464
Epoch:  4  Train Accuracy:  0.98  Test Accuracy:  0.9504
Epoch:  5  Train Accuracy:  0.96  Test Accuracy:  0.9553
Epoch:  6  Train Accuracy:  0.98  Test Accuracy:  0.9582
Epoch:  7  Train Accuracy:  0.94  Test Accuracy:  0.9602
Epoch:  8  Train Accuracy:  0.96  Test Accuracy:  0.9629
Epoch:  9  Train Accuracy:  0.98  Test Accuracy:  0.9637
Epoch:  10  Train Accuracy:  0.98  Test Accuracy:  0.9646
Epoch:  11  Train Accuracy:  0.98  Test Accuracy:  0.966
Epoch:  12  Train Accuracy:  0.96  Test Accuracy:  0.9674
Epoch:  13  Train Accuracy:  0.98  Test Accuracy:  0.9686
Epoch:  14  Train Accuracy:  0.98  Test Accuracy:  0.9693
Epoch:  15  Train Accuracy:  0.98  Test Accuracy:  0.9705
Epoch:  16  Train Accuracy:  1.0  Test Accuracy:  0.9701
Epoch:  17  Train Accuracy:

AND, OR, NOR & NAND Gate Implementation

In [18]:
import numpy as np
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score

# Data for AND Gate
# arr1, target = [[0, 0], [0, 1], [1, 0], [1,1]], [0, 0, 0, 1]

# Data for OR Gate
# arr1, target = [[0, 0], [0, 1], [1, 0], [1,1]], [0, 1, 1, 1]

# Data for NOR Gate
# arr1, target = [[0, 0], [0, 1], [1, 0], [1,1]], [1, 0, 0, 0]

# Data for NAND Gate
arr1, target = [[0, 0], [0, 1], [1, 0], [1,1]], [1, 1, 1, 0]

X = np.array(arr1)
y = np.array(target).astype(np.int)

per_clf = Perceptron(random_state=42, max_iter=100)
per_clf.fit(X, y)

y_pred = per_clf.predict(X)

print(accuracy_score(y, y_pred))

print(per_clf.coef_)
print(per_clf.intercept_)

1.0
[[-2. -3.]]
[4.]


XOR Gate Implementation

In [90]:
import numpy as np
import tensorflow as tf

tf.reset_default_graph()

learning_rate = 0.01
n_epochs = 1000
n_inputs = 2
n_hidden1 = 5
n_outputs = 2

arr1, target = [[0, 0], [0, 1], [1, 0], [1,1]], [0, 1, 1, 0]

X_data = np.array(arr1).astype(np.float32)
y_data = np.array(target).astype(np.int)


X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")


with tf.name_scope("dnn_tf"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    logits = tf.layers.dense(hidden1, n_outputs, name="outputs")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):        
        #sess.run(training_op, feed_dict={X:X_data, y:y_data})    
        #acc_train = accuracy.eval(feed_dict={X:X_data, y:y_data})
        feed_dict={X:X_data, y:y_data}
        t_op, acc_train, t_los, corr = sess.run([training_op, accuracy, loss, correct], feed_dict)
        prediction = [1 if x==y else 0 for (x,y) in zip(corr, target)]
        if acc_train == 1.0: break
        if epoch % 10 == 0:
            print("Epoch: ", epoch, " Training Accuracy: ", acc_train, " Training Loss: ", t_los, 
                 " Prediction: ", prediction)
        
print("Epoch: ", epoch, " Final Accuracy: ", acc_train, " Final Loss: ", t_los, " Final Prediction: ", prediction)


Epoch:  0  Training Accuracy:  0.5  Training Loss:  0.73084766  Prediction:  [0, 0, 0, 0]
Epoch:  10  Training Accuracy:  0.25  Training Loss:  0.69431645  Prediction:  [1, 0, 0, 0]
Epoch:  20  Training Accuracy:  0.75  Training Loss:  0.67686635  Prediction:  [1, 1, 1, 0]
Epoch:  30  Training Accuracy:  0.75  Training Loss:  0.66742486  Prediction:  [1, 1, 1, 0]
Epoch:  40  Training Accuracy:  0.75  Training Loss:  0.65644586  Prediction:  [1, 1, 1, 0]
Epoch:  50  Training Accuracy:  0.75  Training Loss:  0.6445052  Prediction:  [1, 1, 1, 0]
Epoch:  60  Training Accuracy:  0.75  Training Loss:  0.63128364  Prediction:  [1, 1, 1, 0]
Epoch:  70  Training Accuracy:  0.75  Training Loss:  0.61573714  Prediction:  [1, 1, 1, 0]
Epoch:  80  Training Accuracy:  0.5  Training Loss:  0.5979193  Prediction:  [1, 0, 1, 0]
Epoch:  90  Training Accuracy:  0.5  Training Loss:  0.5775427  Prediction:  [1, 0, 1, 0]
Epoch:  100  Training Accuracy:  0.5  Training Loss:  0.5558113  Prediction:  [1, 0, 1,